In [1]:
# !pip install kafka-python

In [2]:
# !pip install tqdm

In [1]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
from json import dumps
from tqdm import tqdm

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
if producer.bootstrap_connected():
    print(f"Successfully connected to bootstrap server")
else:
    print("Couldn't connect to bootstrap server.")

TOPIC_NAME = "ml-raw-dns"

Successfully connected to bootstrap server


In [3]:
def produce_message(producer_instance, topic, message):
    producer_instance.send(topic, message)
    producer_instance.flush()
    return

In [4]:
df = pd.read_csv("test.csv")

df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')

df['label'] = df['label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.to_csv("new_test.csv", index=False)

df.head()

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,53,30449,1,1,49,93,49,49,49.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,443,297054,15,14,851,14141,373,0,56.733333,113.256010,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2,53,75095647,2,2,115,242,58,57,57.500000,0.707107,...,32,50660.0,0.0,50660,50660,74900000.0,0.0,74900000,74900000,0
3,53,209,2,2,90,172,45,45,45.000000,0.000000,...,44,0.0,0.0,0,0,0.0,0.0,0,0,0
4,443,30184469,12,6,859,3157,373,0,71.583333,123.682481,...,32,167703.0,0.0,167703,167703,30000000.0,0.0,30000000,30000000,0


In [5]:
with open("new_test.csv") as f:
    start_time = datetime.now()
    for i, line in tqdm(enumerate(f)):
        produce_message(producer_instance=producer, topic=TOPIC_NAME, message=line)
    end_time = datetime.now()
    print(f"Batch took {end_time-start_time} time for ingesting data")

print("Ingestion Completed")

42022it [01:22, 510.00it/s]

Batch took 0:01:22.455734 time for ingesting data
Ingestion Completed
